In [1]:
 !pip install -r requirements.txt


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from enum import Enum
import seaborn as sns
from pydantic import BaseModel
from dotenv import load_dotenv
import numpy as np # alias
import openai, groq
import pandas as pd # alias

load_dotenv()

False

In [3]:
df = pd.read_csv('data/raw/bookings_train.csv')
df.head(10)

,booking_id,citizen_id,booking_date,appointment_date,appointment_time,check_in_time,check_out_time,task_id,num_documents,queue_number,satisfaction_rating
0,f7b59ba3793fea61282cb02be2247cebd0a75306,4229044420,2021-01-01,2021-01-01,09:03,2021-01-01 09:11:00,2021-01-01 09:48:15.166353269,TASK-002,0,2,4
1,cfe30d53b4b645e4cb6b114f6c9d74a0f25e6a80,5389442635,2021-01-01,2021-01-01,09:12,2021-01-01 09:24:00,2021-01-01 10:24:12.189261137,TASK-001,1,1,4
2,0fb0a5d9b3944dd743a9e7828dc03fa74aa6ebf8,2123783919,2021-01-01,2021-01-01,09:36,2021-01-01 09:29:00,2021-01-01 10:26:48.802260864,TASK-002,1,6,5
3,d4bf826153212fa325ad98b215668ab942365607,6359918966,2021-01-01,2021-01-01,09:45,2021-01-01 10:07:00,2021-01-01 11:00:13.485642822,TASK-001,1,4,4
4,ebb4c9687acbaf65376d8e379bd1341d1c15127e,178732202,2021-01-01,2021-01-01,10:12,2021-01-01 10:26:00,2021-01-01 11:54:53.260180213,TASK-002,1,8,3
5,59611da4309c85c5d1e4e540567a37adadd5f08b,8548814223,2021-01-01,2021-01-01,10:25,2021-01-01 10:25:00,2021-01-01 12:06:10.680457034,TASK-002,4,9,5
6,fa6d423c5467c4b56a5cebda3178210a520d46f1,6467871534,2021-01-01,2021-01-01,10:26,2021-01-01 10:29:00,2021-01-01 11:45:37.549181971,TASK-002,2,2,4
7,2d9c70b44df9735d30ebdf59ba823be155cca1e9,8478849922,2021-01-01,2021-01-01,10:29,2021-01-01 10:35:00,2021-01-01 11:33:33.010326653,TASK-001,0,1,4
8,4a3b77d99250506b937522e9c96098d47db6f2e4,6833251424,2021-01-01,2021-01-01,10:38,2021-01-01 10:34:00,2021-01-01 11:34:08.857731647,TASK-001,3,3,5
9,347039e6bf042ea3a662a46456a19fc24ae329ce,6953076644,2021-01-01,2021-01-01,11:54,NaN,NaN,TASK-002,2,4,4


In [4]:
df_staffing= pd.read_csv('data/raw/staffing_train.csv')
df_staffing.head()

,date,section_id,employees_on_duty,total_task_time_minutes
0,2021-01-01,SEC-001,1,834.569483
1,2021-01-01,SEC-002,4,1525.368990
2,2021-01-01,SEC-003,2,662.095279
3,2021-01-01,SEC-004,4,1499.112620
4,2021-01-01,SEC-005,3,1238.533073


In [5]:
df_tasks= pd.read_csv('data/raw/tasks.csv')
df_tasks.head()

,task_id,task_name,section_id,section_name
0,TASK-001,NaN,SEC-001,NaN
1,TASK-002,NaN,SEC-001,NaN
2,TASK-003,NaN,SEC-002,NaN
3,TASK-004,NaN,SEC-002,NaN
4,TASK-005,NaN,SEC-002,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203693 entries, 0 to 203692
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   booking_id           203693 non-null  object
 1   citizen_id           203693 non-null  int64 
 2   booking_date         203693 non-null  object
 3   appointment_date     203693 non-null  object
 4   appointment_time     203693 non-null  object
 5   check_in_time        197601 non-null  object
 6   check_out_time       197601 non-null  object
 7   task_id              203693 non-null  object
 8   num_documents        203693 non-null  int64 
 9   queue_number         203693 non-null  int64 
 10  satisfaction_rating  203693 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 17.1+ MB


In [7]:
# Check data types
print("Staffing date type:", df_staffing['date'].dtype)
print("Bookings appointment_date type:", df['appointment_date'].dtype)

# Convert dates to same format 
df_staffing['date'] = pd.to_datetime(df_staffing['date'])
df['appointment_date'] = pd.to_datetime(df['appointment_date'])

# Now merge tasks with staffing
df_tasks_staffing = df_tasks.merge(df_staffing, on='section_id', how='left')

# Merge with main dataframe
df = df.merge(
    df_tasks_staffing[['task_id', 'date', 'employees_on_duty']], 
    left_on=['task_id', 'appointment_date'], 
    right_on=['task_id', 'date'], 
    how='left'
)

# Drop the duplicate date column if you don't need it
df.drop('date', axis=1, inplace=True)

# Check results
print(df.head())
print(f"Missing employees_on_duty: {df['employees_on_duty'].isnull().sum()}")

Staffing date type: object
Bookings appointment_date type: object
                                 booking_id  citizen_id booking_date  \
0  f7b59ba3793fea61282cb02be2247cebd0a75306  4229044420   2021-01-01   
1  cfe30d53b4b645e4cb6b114f6c9d74a0f25e6a80  5389442635   2021-01-01   
2  0fb0a5d9b3944dd743a9e7828dc03fa74aa6ebf8  2123783919   2021-01-01   
3  d4bf826153212fa325ad98b215668ab942365607  6359918966   2021-01-01   
4  ebb4c9687acbaf65376d8e379bd1341d1c15127e   178732202   2021-01-01   

  appointment_date appointment_time        check_in_time  \
0       2021-01-01            09:03  2021-01-01 09:11:00   
1       2021-01-01            09:12  2021-01-01 09:24:00   
2       2021-01-01            09:36  2021-01-01 09:29:00   
3       2021-01-01            09:45  2021-01-01 10:07:00   
4       2021-01-01            10:12  2021-01-01 10:26:00   

                  check_out_time   task_id  num_documents  queue_number  \
0  2021-01-01 09:48:15.166353269  TASK-002              0        

### Check for null values

In [8]:
# get null values
df.isnull().sum()

booking_id                0
citizen_id                0
booking_date              0
appointment_date          0
appointment_time          0
check_in_time          6092
check_out_time         6092
task_id                   0
num_documents             0
queue_number              0
satisfaction_rating       0
employees_on_duty         0
dtype: int64

In [9]:
df_staffing.isnull().sum()

date                       0
section_id                 0
employees_on_duty          0
total_task_time_minutes    0
dtype: int64

In [10]:
for col in df.select_dtypes(include='object'):
    df[col] = df[col].replace(r'^\s*$', np.nan, regex=True)
    df[col] = pd.to_numeric(df[col], errors='ignore')
    print(f"{col}: {df[col].isna().sum()} nulls")

booking_id: 0 nulls
booking_date: 0 nulls


/var/folders/lx/fj0jgh0s0rg95r58q33kw2b40000gn/T/ipykernel_28776/473248180.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


appointment_time: 0 nulls
check_in_time: 6092 nulls
check_out_time: 6092 nulls
task_id: 0 nulls


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203693 entries, 0 to 203692
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   booking_id           203693 non-null  object        
 1   citizen_id           203693 non-null  int64         
 2   booking_date         203693 non-null  object        
 3   appointment_date     203693 non-null  datetime64[ns]
 4   appointment_time     203693 non-null  object        
 5   check_in_time        197601 non-null  object        
 6   check_out_time       197601 non-null  object        
 7   task_id              203693 non-null  object        
 8   num_documents        203693 non-null  int64         
 9   queue_number         203693 non-null  int64         
 10  satisfaction_rating  203693 non-null  int64         
 11  employees_on_duty    203693 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 18.6+ MB


In [12]:
df['booking_id'].nunique()

203693

In [13]:
import pandas as pd
import numpy as np

t_in  = pd.to_datetime(df['check_in_time'], errors='coerce')
t_out = pd.to_datetime(df['check_out_time'], errors='coerce')
delta = t_out - t_in

# Round to nearest second, then take full minutes
secs_rounded = delta.dt.total_seconds().round(0)
df['duration_minutes'] = (secs_rounded // 60).astype('Int64')

print(df[['check_in_time','check_out_time','duration_minutes']].head())
print(df['duration_minutes'].describe())

         check_in_time                 check_out_time  duration_minutes
0  2021-01-01 09:11:00  2021-01-01 09:48:15.166353269                37
1  2021-01-01 09:24:00  2021-01-01 10:24:12.189261137                60
2  2021-01-01 09:29:00  2021-01-01 10:26:48.802260864                57
3  2021-01-01 10:07:00  2021-01-01 11:00:13.485642822                53
4  2021-01-01 10:26:00  2021-01-01 11:54:53.260180213                88
count     197601.0
mean     48.317407
std      24.005451
min            5.0
25%           31.0
50%           42.0
75%           59.0
max          217.0
Name: duration_minutes, dtype: Float64


In [14]:
df.drop(columns=['check_in_time','check_out_time'], inplace=True)

df.head(5)

,booking_id,citizen_id,booking_date,appointment_date,appointment_time,task_id,num_documents,queue_number,satisfaction_rating,employees_on_duty,duration_minutes
0,f7b59ba3793fea61282cb02be2247cebd0a75306,4229044420,2021-01-01,2021-01-01,09:03,TASK-002,0,2,4,1,37
1,cfe30d53b4b645e4cb6b114f6c9d74a0f25e6a80,5389442635,2021-01-01,2021-01-01,09:12,TASK-001,1,1,4,1,60
2,0fb0a5d9b3944dd743a9e7828dc03fa74aa6ebf8,2123783919,2021-01-01,2021-01-01,09:36,TASK-002,1,6,5,1,57
3,d4bf826153212fa325ad98b215668ab942365607,6359918966,2021-01-01,2021-01-01,09:45,TASK-001,1,4,4,1,53
4,ebb4c9687acbaf65376d8e379bd1341d1c15127e,178732202,2021-01-01,2021-01-01,10:12,TASK-002,1,8,3,1,88


In [15]:
df.isnull().sum()

booking_id                0
citizen_id                0
booking_date              0
appointment_date          0
appointment_time          0
task_id                   0
num_documents             0
queue_number              0
satisfaction_rating       0
employees_on_duty         0
duration_minutes       6092
dtype: int64

In [16]:

df['appointment_date'] = pd.to_datetime(df['appointment_date'])

### Create bins and new columns to impute the missing values in duration_minutes column

In [17]:
# Extract weekday (0=Monday, 6=Sunday) and month for date patterns
df['weekday'] = df['appointment_date'].dt.weekday
df['month'] = df['appointment_date'].dt.month

# Extract hour from appointment_time
df['appointment_hour'] = pd.to_datetime(df['appointment_time'], format='%H:%M').dt.hour

# Bin hours into categories (e.g., morning: 0-11, afternoon: 12-15, evening: 16+) for coarser grouping
df['time_bin'] = pd.cut(df['appointment_hour'], bins=[0, 12, 15, 24], labels=['morning', 'afternoon', 'evening'], right=False)

In [18]:
# Imputation using Groupby Median
# Define grouping levels 
group_levels =  ['task_id', 'time_bin', 'weekday'] 


In [19]:
# Function to impute 
def impute_duration(grouped_df, levels):
    # Create a copy to avoid modifying original
    df_copy = grouped_df.copy()
    
    # Mask for missing values
    missing_mask = df_copy['duration_minutes'].isnull()
    
    # Compute overall median as ultimate fallback
    overall_median = df_copy['duration_minutes'].median()

    # Group and compute median
    medians = df_copy.groupby(group_levels)['duration_minutes'].median().round().astype('Int64')
    
    # Merge medians back (transform-like)
    df_merged = df_copy.merge(medians.rename('imputed'), on=group_levels, how='left')
    
    to_fill = missing_mask & df_merged['imputed'].notnull()

    # Fill missing where possible
    df_copy.loc[to_fill, 'duration_minutes'] = df_merged.loc[to_fill, 'imputed']
    
    return df_copy

In [20]:
df.head(5)

,booking_id,citizen_id,booking_date,appointment_date,appointment_time,task_id,num_documents,queue_number,satisfaction_rating,employees_on_duty,duration_minutes,weekday,month,appointment_hour,time_bin
0,f7b59ba3793fea61282cb02be2247cebd0a75306,4229044420,2021-01-01,2021-01-01,09:03,TASK-002,0,2,4,1,37,4,1,9,morning
1,cfe30d53b4b645e4cb6b114f6c9d74a0f25e6a80,5389442635,2021-01-01,2021-01-01,09:12,TASK-001,1,1,4,1,60,4,1,9,morning
2,0fb0a5d9b3944dd743a9e7828dc03fa74aa6ebf8,2123783919,2021-01-01,2021-01-01,09:36,TASK-002,1,6,5,1,57,4,1,9,morning
3,d4bf826153212fa325ad98b215668ab942365607,6359918966,2021-01-01,2021-01-01,09:45,TASK-001,1,4,4,1,53,4,1,9,morning
4,ebb4c9687acbaf65376d8e379bd1341d1c15127e,178732202,2021-01-01,2021-01-01,10:12,TASK-002,1,8,3,1,88,4,1,10,morning


In [21]:
# Apply imputation
df = impute_duration(df, group_levels)

print(df.isnull().sum())

booking_id             0
citizen_id             0
booking_date           0
appointment_date       0
appointment_time       0
task_id                0
num_documents          0
queue_number           0
satisfaction_rating    0
employees_on_duty      0
duration_minutes       0
weekday                0
month                  0
appointment_hour       0
time_bin               0
dtype: int64


/var/folders/lx/fj0jgh0s0rg95r58q33kw2b40000gn/T/ipykernel_28776/566822897.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  medians = df_copy.groupby(group_levels)['duration_minutes'].median().round().astype('Int64')


In [22]:
df.head(10)

,booking_id,citizen_id,booking_date,appointment_date,appointment_time,task_id,num_documents,queue_number,satisfaction_rating,employees_on_duty,duration_minutes,weekday,month,appointment_hour,time_bin
0,f7b59ba3793fea61282cb02be2247cebd0a75306,4229044420,2021-01-01,2021-01-01,09:03,TASK-002,0,2,4,1,37,4,1,9,morning
1,cfe30d53b4b645e4cb6b114f6c9d74a0f25e6a80,5389442635,2021-01-01,2021-01-01,09:12,TASK-001,1,1,4,1,60,4,1,9,morning
2,0fb0a5d9b3944dd743a9e7828dc03fa74aa6ebf8,2123783919,2021-01-01,2021-01-01,09:36,TASK-002,1,6,5,1,57,4,1,9,morning
3,d4bf826153212fa325ad98b215668ab942365607,6359918966,2021-01-01,2021-01-01,09:45,TASK-001,1,4,4,1,53,4,1,9,morning
4,ebb4c9687acbaf65376d8e379bd1341d1c15127e,178732202,2021-01-01,2021-01-01,10:12,TASK-002,1,8,3,1,88,4,1,10,morning
5,59611da4309c85c5d1e4e540567a37adadd5f08b,8548814223,2021-01-01,2021-01-01,10:25,TASK-002,4,9,5,1,101,4,1,10,morning
6,fa6d423c5467c4b56a5cebda3178210a520d46f1,6467871534,2021-01-01,2021-01-01,10:26,TASK-002,2,2,4,1,76,4,1,10,morning
7,2d9c70b44df9735d30ebdf59ba823be155cca1e9,8478849922,2021-01-01,2021-01-01,10:29,TASK-001,0,1,4,1,58,4,1,10,morning
8,4a3b77d99250506b937522e9c96098d47db6f2e4,6833251424,2021-01-01,2021-01-01,10:38,TASK-001,3,3,5,1,60,4,1,10,morning
9,347039e6bf042ea3a662a46456a19fc24ae329ce,6953076644,2021-01-01,2021-01-01,11:54,TASK-002,2,4,4,1,64,4,1,11,morning


In [23]:
df.drop(['weekday', 'month', 'appointment_hour', 'time_bin'], axis=1, inplace=True)
df.head()

,booking_id,citizen_id,booking_date,appointment_date,appointment_time,task_id,num_documents,queue_number,satisfaction_rating,employees_on_duty,duration_minutes
0,f7b59ba3793fea61282cb02be2247cebd0a75306,4229044420,2021-01-01,2021-01-01,09:03,TASK-002,0,2,4,1,37
1,cfe30d53b4b645e4cb6b114f6c9d74a0f25e6a80,5389442635,2021-01-01,2021-01-01,09:12,TASK-001,1,1,4,1,60
2,0fb0a5d9b3944dd743a9e7828dc03fa74aa6ebf8,2123783919,2021-01-01,2021-01-01,09:36,TASK-002,1,6,5,1,57
3,d4bf826153212fa325ad98b215668ab942365607,6359918966,2021-01-01,2021-01-01,09:45,TASK-001,1,4,4,1,53
4,ebb4c9687acbaf65376d8e379bd1341d1c15127e,178732202,2021-01-01,2021-01-01,10:12,TASK-002,1,8,3,1,88


In [24]:
df.to_csv('data/processed/bookings_Missingvals_Handled.csv', index=False)